In [50]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
#Run the below piece of code for the first time
#nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [66]:
message_data = pd.read_csv("/content/sample_data/spam.csv",encoding = "latin")
message_data.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,notspam,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,notspam,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,notspam,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,notspam,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [67]:
message_data = message_data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)


In [68]:
message_data = message_data.rename(columns = {'v1':'Spam/Not_Spam','v2':'message'})


In [69]:
message_data.groupby('Spam/Not_Spam').describe()


message         \
                count unique   
Spam/Not_Spam                  
notspam          4825   4516   
spam              747    653   

                                                                       
                                                             top freq  
Spam/Not_Spam                                                          
notspam                                   Sorry, I'll call later   30  
spam           Please call our customer service representativ...    4

In [70]:
message_data_copy = message_data['message'].copy()



In [71]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [72]:
message_data_copy.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: message, dtype: object

In [73]:
message_data_copy = message_data_copy.apply(text_preprocess)
message_data_copy

0       Go jurong point crazy Available bugis n great ...
1                                 Ok lar Joking wif u oni
2       Free entry 2 wkly comp win FA Cup final tkts 2...
3                     U dun say early hor U c already say
4             Nah dont think goes usf lives around though
                              ...                        
5567    2nd time tried 2 contact u U å£750 Pound prize...
5568                          Ì b going esplanade fr home
5569                          Pity mood Soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       Rofl true name
Name: message, Length: 5572, dtype: object

In [74]:
vectorizer = TfidfVectorizer()

In [75]:
message_mat = vectorizer.fit_transform(message_data_copy)
message_mat

<5572x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 47254 stored elements in Compressed Sparse Row format>

In [76]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat,
                                                        message_data['Spam/Not_Spam'], test_size=0.3,
                                                                                    random_state=20)

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9383971291866029